# Extract Transform Load


## Idea general

- 1. La idea general del proyecto es poder generar un Web Scrapin, sencillo para poder genera la extraccion de datos de 2 o 3 paginas web, con contenido sobre los datos disponibles de los equipos de futbol en Mexico ( LigaMX )
- 2. Generar una talba en SQL server o csv, mediante la cual almacenaremos los datos extraidos de la pagina web.
- Las pagina mediante la cual podremos extraer los datos seran:
    - https://www.tudn.com/futbol/liga-mx/resultados
    -  https://www.tudn.com/futbol/liga-mx/posiciones
- 3. Crear una API para poder extrtaer los datos requeridos


## Extract

Dentro del **Extract**, vamos a generar una API para poder conectarnos a la pagina web y posteriormente ,amipularlos.

Los datos que vamos a solicitar, seran los siguientes:

    - Fecha de ejecucion
    - Equipo local
    - Fecha del partido
    - Equipo visitante
    - Goles Local
    - Goles Visitante
    - Estadio del equipo local
    - Hora del juego
    - Ganador
    - Puntos hasta la fecha de la extraccion
    - Tendencia en puntos
    
Los datos los podremos sacar de los soguientes links:

    - https://www.tudn.com/futbol/liga-mx/resultados
    - https://www.tudn.com/futbol/liga-mx/posiciones
    
    
## Transform

En el **Transform** lo que se ejecutara sera la trnasformacion de los datos, como la conversion de la tendencia, los tipos de datos, datos duplicados, fechas, suma de los puntos ( talvez o extraccion directa de la pagina ), entre algunos otros.

## Load.

La carga se podra generar en 2 formas.

La primera sera, guardandolo en un csv.

La segunda sera, almacenandola en sql server ( Talvez mysql ).




Es possible que en transcurso de la creacion, agrege mas variables, las cuales se adjuntaran dentro de este documento de presentacion.

La tabla, podra quedar de la siguiente forma.

In [18]:
import pandas as pd

df = pd.DataFrame()

df['Date_Flow'] = ['2022-01-7', '2022-01-10']
df['Team_Local'] = ['Puebla', 'Chivas']
df['Date_match'] = ['2022-01-7', '2022-01-09']
df['Team_Visit'] = ['America', 'Tigres']
df['Goals_Local'] = ['5', '0']
df['Goals_Visit'] = ['0', '2']
df['Estadium_team'] = ['Puebla', 'Chivas']
df['Hour_play'] = ['8:00 PM', '9:00 PM']
df['Win'] = ['Puebla', 'Tigres']
df['Puntos'] = ['3', '3']
df['Trend'] = ['+', '+']



df.head()

,Date_Flow,Team_Local,Date_match,Team_Visit,Goals_Local,Goals_Visit,Estadium_team,Hour_play,Win,Puntos,Trend
0,2022-01-7,Puebla,2022-01-7,America,5,0,Puebla,8:00 PM,Puebla,3,+
1,2022-01-10,Chivas,2022-01-09,Tigres,0,2,Chivas,9:00 PM,Tigres,3,+


In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.tudn.com/futbol/liga-mx/resultados'
html = requests.get(url)

In [5]:
#Extraer las etiquetas del HTLMK y su contenido
soup = BeautifulSoup(html.content, 'html.parser')


In [49]:
time = soup.find_all("p", {"class": "TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-regular DateTime__DateTimeStyled-sc-frci42-0 kOdh _31Klo"})
time

[<p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-regular DateTime__DateTimeStyled-sc-frci42-0 kOdh _31Klo">Sábado, 23 de Julio de 2022</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-regular DateTime__DateTimeStyled-sc-frci42-0 kOdh _31Klo">Domingo, 24 de Julio de 2022</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-regular DateTime__DateTimeStyled-sc-frci42-0 kOdh _31Klo">Lunes, 25 de Julio de 2022</p>]

In [43]:
local = soup.find_all("p", {"class": "TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl"})
local

[<p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl">Mazatlán FC</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl">Necaxa</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl">Toluca</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl">Cruz Azul</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl">Club Tijuana</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl">Tigres</p>,
 <p class="TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl">Pachuca</p>]

In [84]:
""" Obtenemos equipos locales y visitantes"""

teams_all={'Local':[],'Visitante':[]}
for team in range (len(local) ):
    team_local = soup.find_all("p", {"class": "TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 ehCibl"})[team].text.strip()
    teams_visitante = soup.find_all("p", {"class": "TextUVN__TextStyled-sc-1i54lfj-0 cLBXKg uvs-font-a-bold Team__TeamName-sc-1so5tpb-0 jfaVYh"})[team].text.strip()
    
    teams_all['Local'].append(team_local)
    teams_all['Visitante'].append(teams_visitante)

In [85]:
teams_all

{'Local': ['Mazatlán FC',
  'Necaxa',
  'Toluca',
  'Cruz Azul',
  'Club Tijuana',
  'Tigres',
  'Pachuca'],
 'Visitante': ['Atlético San Luis',
  'FC Juárez',
  'Santos Laguna',
  'Puebla',
  'América',
  'Atlas',
  'Pumas UNAM']}

In [97]:
teams_result={'Goals_Local':[],'Goals_Visit':[]}
for team in range (len(local) ):
    try:
        result_local = soup.find_all("div", {"class": "TeamScore__ScoreValueWrapper-sc-1bfk9ze-1 bdVkOu rn-1oszu61 rn-1efd50x rn-14skgim rn-rull8r rn-mm0ijv rn-13yce4e rn-fnigne rn-ndvcnb rn-gxnn5r rn-deolkf rn-6koalj rn-1qe8dj5 rn-1mlwlqe rn-eqz5dr rn-1mnahxq rn-61z16t rn-p1pxzi rn-11wrixw rn-ifefl9 rn-bcqeeo rn-wk8lta rn-9aemit rn-1mdbw0j rn-gy4na3 rn-bnwqim rn-1lgpqti"})[team].text.strip()
        result_visit = soup.find_all("div", {"class": "TeamScore__ScoreValueWrapper-sc-1bfk9ze-1 iGHBvL rn-1oszu61 rn-1efd50x rn-14skgim rn-rull8r rn-mm0ijv rn-13yce4e rn-fnigne rn-ndvcnb rn-gxnn5r rn-deolkf rn-6koalj rn-1qe8dj5 rn-1mlwlqe rn-eqz5dr rn-1mnahxq rn-61z16t rn-p1pxzi rn-11wrixw rn-ifefl9 rn-bcqeeo rn-wk8lta rn-9aemit rn-1mdbw0j rn-gy4na3 rn-bnwqim rn-1lgpqti"})[team].text.strip()
        teams_result['Goals_Local'].append(result_local)
        teams_result['Goals_Visit'].append(result_visit)
    except:
        teams_result['Goals_Local'].append(0)
        teams_result['Goals_Visit'].append(0)
    
teams_result

{'Goals_Local': ['1', '1', '2', '2', '2', '2', 0],
 'Goals_Visit': ['1', '0', '1', '2', '0', '0', 0]}